In [7]:
import os, getpass
import asyncio
import nest_asyncio
from typing import List
from dotenv import load_dotenv, find_dotenv
import logging

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool as langchain_tool
from langchain.agents import create_agent

# UNCOMMENT
# Prompt the user securely and set API keys as an environment variables
load_dotenv(find_dotenv())

try:
   # A model with function/tool calling capabilities is required.
   llm = ChatOpenAI(model="gemini-2.5-flash", temperature=0, base_url=os.getenv("OPENAI_API_BASE"), api_key=os.getenv("OPENAI_API_KEY"))
   print(f"✅ Language model initialized: {llm.model_name}")
except Exception as e:
   print(f"🛑 Error initializing language model: {e}")
   llm = None

# --- Define a Tool ---
@langchain_tool
def search_information(query: str) -> str:
   """
   Provides factual information on a given topic. Use this tool to find answers to phrases
   like 'capital of France' or 'weather in London?'.
   """
   print(f"\n--- 🛠️ Tool Called: search_information with query: '{query}' ---")
   # Simulate a search tool with a dictionary of predefined results.
   simulated_results = {
       "weather in london": "The weather in London is currently cloudy with a temperature of 15°C.",
       "capital of france": "The capital of France is Paris.",
       "population of earth": "The estimated population of Earth is around 8 billion people.",
       "tallest mountain": "Mount Everest is the tallest mountain above sea level.",
       "default": f"Simulated search result for '{query}': No specific information found, but the topic seems interesting."
   }
   result = simulated_results.get(query.lower(), simulated_results["default"])
   print(f"--- TOOL RESULT: {result} ---")
   return result

tools = [search_information]

# --- Create a Tool-Calling Agent ---
if llm:
   # Create the agent using the new create_agent API
   agent = create_agent(
       model=llm,
       tools=tools,
       system_prompt="You are a helpful assistant."
   )

async def run_agent_with_tool(query: str):
   """Invokes the agent with a query and prints the final response."""
   print(f"\n--- 🏃 Running Agent with Query: '{query}' ---")
   try:
       # The new agent expects a dictionary with 'messages'
       response = await agent.ainvoke({"messages": [{"role": "user", "content": query}]})
       print("\n--- ✅ Final Agent Response ---")
       # The response contains the full state, we want the content of the last message
       print(response["messages"][-1].content)
   except Exception as e:
       print(f"\n🛑 An error occurred during agent execution: {e}")

async def main():
   """Runs all agent queries concurrently."""
   tasks = [
       run_agent_with_tool("What is the capital of France?"),
       run_agent_with_tool("What's the weather like in London?"),
       run_agent_with_tool("Tell me something about dogs.") # Should trigger the default tool response
   ]
   await asyncio.gather(*tasks)

nest_asyncio.apply()
asyncio.run(main())

✅ Language model initialized: gemini-2.5-flash

--- 🏃 Running Agent with Query: 'What is the capital of France?' ---

--- 🏃 Running Agent with Query: 'What's the weather like in London?' ---

--- 🏃 Running Agent with Query: 'Tell me something about dogs.' ---

--- 🛠️ Tool Called: search_information with query: 'weather in London' ---
--- TOOL RESULT: The weather in London is currently cloudy with a temperature of 15°C. ---

--- 🛠️ Tool Called: search_information with query: 'dogs' ---
--- TOOL RESULT: Simulated search result for 'dogs': No specific information found, but the topic seems interesting. ---

--- 🛠️ Tool Called: search_information with query: 'capital of France' ---
--- TOOL RESULT: The capital of France is Paris. ---

--- ✅ Final Agent Response ---
The weather in London is currently cloudy with a temperature of 15°C.

--- ✅ Final Agent Response ---
The capital of France is Paris.

--- ✅ Final Agent Response ---
It seems I couldn't find specific information about dogs using